<a href="https://colab.research.google.com/github/minyoung347/python101/blob/main/prediction_sp500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Pandas 시작- 파일을 DataFrame 로딩, 기본 API

In [1]:
import pandas as pd

In [ ]:
!rm -r python101/
!git clone https://github.com/minyoung347/python101.git

Cloning into 'python101'...
remote: Enumerating objects: 785, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 785 (delta 160), reused 287 (delta 144), pack-reused 463
Receiving objects: 100% (785/785), 91.96 MiB | 36.25 MiB/s, done.
Resolving deltas: 100% (401/401), done.


In [ ]:
df = pd.read_csv('/content/python101/input/SP500HistoricalData.csv')

In [ ]:
def remove_comma_float(x):
    return float(x.replace(',', ''))

In [ ]:
df['Price'] = df['Price'].apply(remove_comma_float)
df['Open'] = df['Open'].apply(remove_comma_float)
df['High'] = df['High'].apply(remove_comma_float)
df['Low'] = df['Low'].apply(remove_comma_float)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [ ]:
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

In [ ]:
df.drop('Vol.', inplace=True, axis=1)

In [ ]:
df['Price_diff'] = df['Price'].diff(1)

In [ ]:
df['up_down'] = 0

In [ ]:
# up: 1, down: 0

In [ ]:
df.loc[df.Price_diff > 0, 'up_down'] = 1

In [ ]:
df['up_down_shift'] = df['up_down'].shift(-1)

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
df = df.astype({'up_down_shift': 'int'})

In [ ]:
df['up_down_shift_minus_1'] = df['up_down'].shift(-1)
df['up_down_shift_plus_1'] = df['up_down'].shift(1)

In [ ]:
df.drop(['up_down_shift_minus_1', 'up_down_shift_plus_1'], axis=1, inplace=True)

In [ ]:
df

,Price,Open,High,Low,Change %,Price_diff,up_down,up_down_shift
Date,,,,,,,,
2019-01-03,2447.89,2491.92,2493.14,2443.96,-2.48,-62.14,0,1
2019-01-04,2531.94,2474.33,2538.07,2474.33,3.43,84.05,1,1
2019-01-07,2549.69,2535.61,2566.16,2524.56,0.70,17.75,1,1
2019-01-08,2574.41,2568.11,2579.82,2547.56,0.97,24.72,1,1
2019-01-09,2584.96,2580.00,2595.32,2568.89,0.41,10.55,1,1
...,...,...,...,...,...,...,...,...
2023-06-13,4369.01,4352.61,4375.37,4349.31,0.69,30.08,1,1
2023-06-14,4372.59,4366.29,4391.82,4337.85,0.08,3.58,1,1
2023-06-15,4425.84,4365.33,4439.20,4362.60,1.22,53.25,1,0


In [ ]:
def strip_pct_float(x):
    return float(x.replace('%', ''))

In [ ]:
df['Change %'] = df['Change %'].apply(strip_pct_float)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1123 entries, 2019-01-03 to 2023-06-20
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Price          1123 non-null   float64
 1   Open           1123 non-null   float64
 2   High           1123 non-null   float64
 3   Low            1123 non-null   float64
 4   Change %       1123 non-null   float64
 5   Price_diff     1123 non-null   float64
 6   up_down        1123 non-null   int64  
 7   up_down_shift  1123 non-null   int64  
dtypes: float64(6), int64(2)
memory usage: 79.0 KB


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['up_down', 'up_down_shift'], axis=1)

In [ ]:
y = df['up_down_shift']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, shuffle=False)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print('--------------')
print(X_test.shape)
print(y_test.shape)

(898, 6)
(898,)
--------------
(225, 6)
(225,)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

# 결정트리, Random Forest, 로지스틱 회귀를 위한 사이킷런 Classifier 클래스 생성
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train , y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train , y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifier 정확도:{0:.4f}'.format(accuracy_score(y_test, rf_pred)))

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train , y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test, lr_pred)))


DecisionTreeClassifier 정확도: 0.5244
RandomForestClassifier 정확도:0.4978
LogisticRegression 정확도: 0.4756
